# Compare different prompts to extract frames from climate news

In [1]:
import buttermilk

bm = buttermilk.BM()

from buttermilk.flows.extract import Analyst

/opt/conda/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
flow = Analyst(prompt_template_path="generic.prompty", langchain_model_name="haiku")
flow.template

JinjaTemplateError: Failed to render jinja template: TypeError: no loader for this environment specified. Please modify your prompt to fix the issue.